<a href="https://colab.research.google.com/github/DivyaDharshini2293/Deep-Learning/blob/main/BYTS_Keras_VS_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import tensorflow as tf
from  torch.utils.data import DataLoader
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [20]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
train_dataset=MNIST(root="./data",train=True,download=True,transform=transform)
test_dataset=MNIST(root="./data",train=False,download=True,transform=transform)

In [21]:
train_loader=DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=64,shuffle=False)

In [22]:
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

In [23]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [24]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [25]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.flatten=nn.Flatten()
    self.fc1=nn.Linear(28*28,128)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(128,10)
    self.softmax=nn.Softmax(dim=1)
  def forward(self,x):
    x=self.flatten(x)
    x=self.fc1(x)
    x=self.relu(x)
    x=self.fc2(x)
    x=self.softmax(x)
    return x

In [26]:
model_pt=Net()
criterion=nn.CrossEntropyLoss()
optimizer_pt=optim.Adam(model_pt.parameters(),lr=0.001)


In [27]:
import time
start_time=time.time()
model_tf=models.Sequential()
model_tf.add(layers.Flatten(input_shape=(28,28,1)))
model_tf.add(layers.Dense(128,activation='relu'))
model_tf.add(layers.Dense(10,activation='softmax'))

In [28]:
model_tf.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [29]:
model_tf.fit(train_images,train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time=time.time()

Epoch 1/5
750/750 [==============================] - 4s 5ms/step - loss: 0.3291 - accuracy: 0.9079 - val_loss: 0.1748 - val_accuracy: 0.9508
Epoch 2/5
750/750 [==============================] - 4s 5ms/step - loss: 0.1542 - accuracy: 0.9558 - val_loss: 0.1360 - val_accuracy: 0.9607
Epoch 3/5
750/750 [==============================] - 4s 6ms/step - loss: 0.1095 - accuracy: 0.9682 - val_loss: 0.1082 - val_accuracy: 0.9685
Epoch 4/5
750/750 [==============================] - 4s 5ms/step - loss: 0.0829 - accuracy: 0.9759 - val_loss: 0.1013 - val_accuracy: 0.9701
Epoch 5/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0665 - accuracy: 0.9808 - val_loss: 0.1001 - val_accuracy: 0.9708


In [30]:
print(f"Tensorflow Training Time: {end_time -start_time} seconds")

Tensorflow Training Time: 21.77545976638794 seconds


In [31]:
start_time_p=time.time()
for epoch in range(5):
  for images, labels in train_loader:
    optimizer_pt.zero_grad()
    outputs=model_pt(images.view(-1,28*28))
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer_pt.step()
end_time_p=time.time()
print(f"pytorch training time : {end_time_p -start_time_p} seconds")


pytorch training time : 100.40101957321167 seconds


In [32]:
test_loss_tf,test_acc_tf=model_tf.evaluate(test_images,test_labels)
print(f"Accuracy of Tensorflow :{test_acc_tf}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0979 - accuracy: 0.9703
Accuracy of Tensorflow :0.970300018787384


In [33]:
correct=0
total=0
with torch.no_grad():
  for images, labels in test_loader:
    outputs=model_pt(images.view(-1,28*28))
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted == labels).sum().item()
accuracy_pt=correct/total
print(f"Accuracy of PyTorch {accuracy_pt}")

Accuracy of PyTorch 0.9528
